<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [23]</a>'.</span>

In [29]:
################## This code is a sim2l designed to be used with NEMO1d ##################
#
#  Version 0.0


In [30]:
import json
from simtool import DB
import math
import simtool as st
from hublib.cmd import runCommand  # type: ignore
import uuid
import os
import shutil
import nbformat # type: ignore 
import numpy as np # type: ignore
import plotly.graph_objects as go # type: ignore
import ipywidgets as w # type: ignore
from ipywidgets import HBox, VBox, HTML, Image, Layout, Button, ButtonStyle, Tab, BoundedIntText, BoundedFloatText, Dropdown, ToggleButton, Output, Textarea, FloatLogSlider # type: ignore
%load_ext yamlmagic

The yamlmagic extension is already loaded. To reload it, use:
  %reload_ext yamlmagic


In [31]:
# Set debug mode: 1 to enable, 0 to disable
# Debug mode will enable specific print statements and will only adjust the temperature input 

debug_mode = 1

In [32]:
%%yaml INPUTS
Energy_Band_Model: # not needed
    type: Choice
    description: Energy Band Model
    options: ['Effective Mass Model', 'Tight-Binding Sp3s* Model', 'Tight-Binding sp3d5s* Model']
    value: Effective Mass Model

Potential_model:
    type: Choice
    description: Potential Model
    options: ['Thomas-Fermi', 'Hartree']
    value: Thomas-Fermi

Ending_Bias:
    type: Number
    description: Ending Bias
    max: 1000
    min: 0
    value: 0

Device:
    type: Choice
    description: Device Type
    options: ['1-Barrier-Device', '2-Barrier-Device', '3-Barrier-Device', '6-Barrier-Device']
    value: 2-Barrier-Device

points:
    type: Number
    description: No. of Points
    max: 100.0
    min: 1.0
    value: 6

Starting_bias:
    type: Number
    description: Starting Bias
    max: 100.0
    min: 0.0
    value: 0

TEMPERATURE:
    type: Number
    description: Temperature
    max: 500.0
    min: 77.0


SC_ChargeRegion: #min and max any place with regions
    type: Number
    description: Semiclassical Charge Region dSC (nm)
    min: 0
    value: 0

EQ_Region:
    type: Number
    description: Equilibrium Region dEQ (nm)
    min: 0
    value: 0

Quantum_charge:
    type: Boolean
    description: Quantum Charge
    value: false

Lattice_Constant:
    type: Number
    description: Lattice Constant (nm)
    value: 0.283

Not_Normalized_Current_Plot:
    type: Boolean
    description: Not-Normalized Current Plot
    value: false

Scatter_Plot:
    type: Boolean
    description: Scatter Plot
    value: false

Carrier_Surface_Distrubution_Plot:
    type: Boolean
    description: Carrier Surface Distribution Plot
    value: false

Resivoir_Relaxation_Model:
    type: Choice
    description: Reservoir Relaxation Model
    options: ['Energy Independant', 'Exponentialy Damped', 'Lorentzian Damped']
    value: Energy Independant

Resivoir_Relaxation_Energy:
    type: Number
    description: Reservoir Relaxation Energy (meV)
    min: 0
    value: 0

Decay_Length:
    type: Number
    description: Decay Length (meV)
    min: 0
    value: 0

Resonance_Finder:
    type: Boolean
    description: Resonance Finder
    value: false

No_Homogenous_Grid_Points:
    type: Number
    description: No. Homogeneous Grid Points
    min: 0
    value: 0

No_Points_per_res:
    type: Number
    description: No. Points Per Resonance
    min: 0
    value: 0

No_Points_per_EC_EF:
    type: Number
    description: No. Points Per EC EF
    min: 0
    value: 0

Lancos_Iteration_Step_Size:
    type: Number
    description: Lanczos Iteration Step Size (eV)
    min: 0
    value: 0

Lancos_Iteration_Limit:
    type: Number
    description: Lanczos Iteration Limit
    min: 0
    value: 0

Newton_Iteration_Step_Size:
    type: Number
    description: Newton Iteration Step Size (eV)
    min: 0
    value: 0

Newton_Solver_Convergence_Condition:
    type: Number
    description: Newton Solver Convergence Condition (eV)
    value: 0


<IPython.core.display.Javascript object>

In [33]:
# From the sim2l library 
# Ensures that the values passed are within specified range 
# Uses the default inputs if nothing is passed



parameters = st.utils.getValidatedInputs(INPUTS)  #type= ignore 


if parameters:
    globals().update(parameters)

#print(parameters)
    

In [34]:
# Parameters
Energy_Band_Model = "Effective Mass Model"
Potential_model = "Thomas-Fermi"
Ending_Bias = 0
Device = "2-Barrier-Device"
points = 6.0
Starting_bias = 0.0
TEMPERATURE = 300.0
SC_ChargeRegion = 0
EQ_Region = 0
Quantum_charge = False
Lattice_Constant = 0
Not_Normalized_Current_Plot = False
Scatter_Plot = False
Carrier_Surface_Distrubution_Plot = False
Resivoir_Relaxation_Model = "Energy Independant"
Resivoir_Relaxation_Energy = 0
Decay_Length = 0
Resonance_Finder = False
No_Homogenous_Grid_Points = 0
No_Points_per_res = 0
No_Points_per_EC_EF = 0
Lancos_Iteration_Step_Size = 0
Lancos_Iteration_Limit = 0
Newton_Iteration_Step_Size = 0
Newton_Solver_Convergence_Condition = 0


In [35]:
# Takes injected parameters and converts it to a dictionary 
# looks through the notebook for a cell with tag injected-paramaters
# Once found it converts the injected parameters to a dictionary and updates the parameters dictionary

def convert_parameters_to_dict(tag):
    # Get the current directory
    current_directory = os.getcwd()
    
    # Define the notebook file name
    notebook_file = os.path.join(current_directory, 'nemo1dsim2l.ipynb')

    # Read the notebook file using nbformat 
    try:
        with open(notebook_file, 'r', encoding='utf-8') as f:
            notebook = nbformat.read(f, as_version=4)
    except Exception as e:
        print(f"Error reading notebook file: {e}")
        return {}

    # Initialize an empty dictionary to hold the parameters
    parameters = {}

    # Loop through each cell in the notebook
    for cell in notebook.cells:
        # Check if the cell has metadata and tags field
        if 'metadata' in cell and 'tags' in cell.metadata:
            # If the tag is in the cell's metadata, process the content
            if tag in cell.metadata['tags']:
                print(f"Found cell with tag '{tag}':")
                print(cell.source)  # Print the content of the cell for debugging
                
                # Extract the content of the cell
                cell_content = cell.source
                
                # Split the content into lines and parse each line
                for line in cell_content.split('\n'):
                    if '=' in line:
                        # Split each line into variable name and value
                        var_name, var_value = line.split('=', 1)
                        var_name = var_name.strip()
                        var_value = var_value.strip()
                        
                        # Attempt to parse the value as an integer, float, or boolean
                        if var_value.lower() in ['true', 'false']:
                            parameters[var_name] = var_value.lower() == 'true'
                        elif var_value.replace('.', '', 1).isdigit():  # Check if it's a float
                            parameters[var_name] = float(var_value)
                        else:
                            parameters[var_name] = var_value.strip('"')  # Remove quotes for strings

    return parameters
parameters = convert_parameters_to_dict('injected-parameters')


Found cell with tag 'injected-parameters':
# Parameters
Energy_Band_Model = "Effective Mass Model"
Potential_model = "Thomas-Fermi"
Ending_Bias = 0
Device = "2-Barrier-Device"
points = 6.0
Starting_bias = 0.0
TEMPERATURE = 300.0
SC_ChargeRegion = 0
EQ_Region = 0
Quantum_charge = False
Lattice_Constant = 0
Not_Normalized_Current_Plot = False
Scatter_Plot = False
Carrier_Surface_Distrubution_Plot = False
Resivoir_Relaxation_Model = "Energy Independant"
Resivoir_Relaxation_Energy = 0
Decay_Length = 0
Resonance_Finder = False
No_Homogenous_Grid_Points = 0
No_Points_per_res = 0
No_Points_per_EC_EF = 0
Lancos_Iteration_Step_Size = 0
Lancos_Iteration_Limit = 0
Newton_Iteration_Step_Size = 0
Newton_Solver_Convergence_Condition = 0



In [36]:
%%yaml OUTPUTS

NEMO1D Input Deck:
    type: File    
    description : NEMO1D Input Deck
IV:
    type : Dict
    description : IV output file
Band:
    type: Dict
    description: Test
    

<IPython.core.display.Javascript object>

In [37]:
# Creates the data base for the outputs 
# Creates a unique identifer for each newinput deck

db = DB(OUTPUTS)  #type: ignore


Newinputdeck= str(uuid.uuid4().hex)

In [38]:
# Path to Material File and Input Deck
# Needed for current implementation
# In the future may need to have these in a local directory where the tool is hosted

testfile = '/home/toro0/NEMO1D-RTD/simtool/test.nem'
material_file = '/home/toro0/NEMO1D-RTD/simtool/GaAs.mat'
band_file =  '/home/toro0/NEMO1D-RTD/simtool/test_AlGaAs_TFQ_Cgam.nd_bnd'
IV_file =  '/home/toro0/NEMO1D-RTD/simtool/IVtest'


if debug_mode == 1:
    print("Debug Mode Active")

print("Parameters before input deck creation:", parameters)

Debug Mode Active
Parameters before input deck creation: {'Energy_Band_Model': 'Effective Mass Model', 'Potential_model': 'Thomas-Fermi', 'Ending_Bias': 0.0, 'Device': '2-Barrier-Device', 'points': 6.0, 'Starting_bias': 0.0, 'TEMPERATURE': 300.0, 'SC_ChargeRegion': 0.0, 'EQ_Region': 0.0, 'Quantum_charge': False, 'Lattice_Constant': 0.0, 'Not_Normalized_Current_Plot': False, 'Scatter_Plot': False, 'Carrier_Surface_Distrubution_Plot': False, 'Resivoir_Relaxation_Model': 'Energy Independant', 'Resivoir_Relaxation_Energy': 0.0, 'Decay_Length': 0.0, 'Resonance_Finder': False, 'No_Homogenous_Grid_Points': 0.0, 'No_Points_per_res': 0.0, 'No_Points_per_EC_EF': 0.0, 'Lancos_Iteration_Step_Size': 0.0, 'Lancos_Iteration_Limit': 0.0, 'Newton_Iteration_Step_Size': 0.0, 'Newton_Solver_Convergence_Condition': 0.0}


In [39]:
def copy_Band_file(destination_dir): ### Remove in final iteration
  
    dest_material_file = os.path.join(destination_dir, os.path.basename(band_file))
  
    if not os.path.exists(dest_material_file):
        shutil.copy2(band_file, dest_material_file)
        print(f"Copied {band_file} to {destination_dir}")
  
    else:
        print(f"{band_file} already exists in {destination_dir}")
current_dir = os.getcwd()
copy_Band_file(current_dir)

/home/toro0/NEMO1D-RTD/simtool/test_AlGaAs_TFQ_Cgam.nd_bnd already exists in /home/toro0/NEMO1D-RTD/simtool/RUNS/dfd86abe0a8f42f2bd560f7807f460c1


In [40]:
# Copies the desired material file to the Directory where the sim2l is run
# Takes the destination directory as an input 
# The material file is defined as a global variable above

def copy_material_file(destination_dir):
  
    dest_material_file = os.path.join(destination_dir, os.path.basename(material_file))
  
    if not os.path.exists(dest_material_file):
        shutil.copy2(material_file, dest_material_file)
        print(f"Copied {material_file} to {destination_dir}")
  
    else:
        print(f"{material_file} already exists in {destination_dir}")

In [41]:
# Reads the input deck file given in the argument and reads it 

def read_input_deck(filepath):
    
    with open(filepath, 'r') as file:
        return file.read()

In [42]:
# Replaces the values from parameters and puts them in the new input deck
# Debug mode makes it to wher only temperature is changed 
# To change the value affected change the key variable

debug_mode == 1
print(debug_mode)
def replace_placeholders(content, parameters):
    ""
    if debug_mode == 1:
        # Only replace the TEMPERATURE placeholder
        key = "TEMPERATURE"
        if key in parameters:
            content = content.replace(key, str(parameters[key]))
            print(f"Replacing {key} with {parameters[key]}")
    else:
        # Replace all placeholders
        for key, value in parameters.items():
            content = content.replace(key, str(value))
            print(f"Replacing {key} with {value}")
    return content

1


In [43]:
# Writes the new input deck with the values from the dictionary
# Takes the new paraamters new_content and output file name as arguments 
# Returns the input deck with injected parameters

def write_new_input_deck(new_content, output_filename):
    with open(output_filename, 'w') as file:
        file.write(new_content)
    print(f"New input deck written to {output_filename}")
    return output_filename

In [44]:
# Copies all the required files and returns a new input deck 
# This function combines other functions to generate a new input deck based on the injected parameters
# Returns the new input deck

def inputdeckCreator():
 
    current_dir = os.getcwd()

    copy_material_file(current_dir)

    content = read_input_deck(testfile)

    updated_content = replace_placeholders(content, parameters)
    
    new_input_deck_filename = 'new_input_deck.nem'

    write_new_input_deck(updated_content, new_input_deck_filename)

    if debug_mode == 1:
        print(f"Debug mode: Simulating with input deck {new_input_deck_filename}")

    return new_input_deck_filename


In [45]:
# Delete later this is a test
# 


def generate_bjt_bandstructure_with_bands(filename):
    # Generate synthetic data for BJT bandstructure
    # x_values could represent the wave vector (k) ranging from -pi to pi
    x_values = np.linspace(-np.pi, np.pi, 100)
    
    # Valence band (parabolic, lower energy)
    y_values_valence = -0.1 * x_values**2 + 0.5  # Example for valence band (negative curvature)
    
    # Conduction band (parabolic, higher energy)
    y_values_conduction = 0.1 * x_values**2 + 2.0  # Example for conduction band (positive curvature)
    
    # Fermi level (constant)
    fermi_level = 1.5
    
    # Save the data to a file
    with open(filename, 'w') as file:
        # Write valence band data
        for x, y in zip(x_values, y_values_valence):
            file.write(f"{x} {y} {fermi_level}\n")  # Add Fermi level for each point
        
        # Write conduction band data
        for x, y in zip(x_values, y_values_conduction):
            file.write(f"{x} {y} {fermi_level}\n")  # Add Fermi level for each point

    print(f"Data saved to {filename}")

In [46]:

# Reads a JSON File output from NEMO1D
# Returns a Dictionary containing the data

def read_data_from_json_file(filename):
 

    with open(filename, 'r') as file:
        # Read the entire content of the file
        content = file.read()
    
    # Parse the content using json.loads (expects valid JSON format)
    try:
        data = json.loads(content)
        print(type(data))
        return data
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON file: {e}")
        return None

In [47]:
# Reads the data for the IV file will potentialy need to be renamed 
# Will be removed in final deployment 
# Use the read from JSON function instead

def read_data_from_file(filename):
    # Initialize lists to store X and Y values
    x_values = []
    y_values = []

    # Read the file line by line
    with open(filename, 'r') as file:
        for line in file:
            # Split each line into X and Y values based on space
            parts = line.split()
            if len(parts) == 2:
                # Convert to float and store in the lists
                x_values.append(float(parts[0]))
                y_values.append(float(parts[1]))

    return tuple(x_values), tuple(y_values)


In [48]:
# Delete later
# Test that will not be used in final deployment

def read_data_from_bjtfile(filename):
   
    data_dict = {
        'x_values': [],
        'valence_band': [],
        'conduction_band': [],
        'fermi_level': []
    }

    # Read the file line by line
    with open(filename, 'r') as file:
        for line in file:
            # Split each line into X, Y (energy), and Fermi level
            parts = line.split()
            if len(parts) == 3:
                # Append data to the dictionary lists
                data_dict['x_values'].append(float(parts[0]))
                data_dict['valence_band'].append(float(parts[1]))
                data_dict['conduction_band'].append(float(parts[2]))  # Fermi level
                data_dict['fermi_level'].append(float(parts[2]))  # Same for both bands
    
    return data_dict

In [49]:
# Calls the input deck creator to create the new input deck 
# Stores the input deck to the database

inputdeckCreator()
db.save("NEMO1D Input Deck", file='new_input_deck.nem') 

/home/toro0/NEMO1D-RTD/simtool/GaAs.mat already exists in /home/toro0/NEMO1D-RTD/simtool/RUNS/dfd86abe0a8f42f2bd560f7807f460c1
Replacing TEMPERATURE with 300.0
New input deck written to new_input_deck.nem
Debug mode: Simulating with input deck new_input_deck.nem


In [50]:
# Runs the simulation Engine  with the new input deck 
# Will need to replace path wiht directory of binary in final deployment

runCommand('/home/toro0/NEMO1D-RTD/simtool/negf new_input_deck.nem')


system_start WARNING: The NEMOHOME environmental variable is not set
NEMO will assume "/usr/local/nemo" is the correct home directory.
No memory report in this platform.
Open File: new_input_deck.nem, 
Initialize the basic device structure arrays.
Update Structural Device Parameters and Initialize Arrays
Copy the material information to the individual monolayers.
Copy Material Parameters into Device Arrays
Finished Device Structure Update
Process Simulation Parameter Window: Calculation Mode
Read and Initialize Simulation Parameter Window: I-V Parameters
Process Simulation Parameter Window: I-V Parameters
Read and Initialize Simulation Parameter Window: Output Options for I-V Calculation
Process Simulation Parameter Window: Output Options for I-V Calculation
Read and Initialize Simulation Parameter Window: Output Options for I-V Calculation
Process Simulation Parameter Window: Output Options for I-V Calculation
Read and Initialize Simulation Parameter Window: Quantum Region
Process Sim

Starting IV scan...
Bias: 0.000000
read_struct Warning: Tried to read a structure
delimited by:
<begin_sheet_region>
<end_sheet_region>
but could not find it.
Skip over the reading of this structure.
Bias: 0.020000
Bias: 0.040000
Bias: 0.060000
Bias: 0.080000
Bias: 0.100000
Bias: 0.120000
Bias: 0.140000
Bias: 0.160000
Bias: 0.180000
Bias: 0.200000
Bias: 0.220000
Bias: 0.240000
Bias: 0.260000
Bias: 0.280000
Bias: 0.300000
Bias: 0.320000
Bias: 0.320000 current integr. fixed e-grid exam res.
Bias: 0.340000
Bias: 0.360000
Bias: 0.380000
Bias: 0.400000
Bias: 0.420000
Bias: 0.440000
Bias: 0.460000
Bias: 0.480000
Bias: 0.500000
Bias: 0.520000
Bias: 0.540000
Bias: 0.560000
Bias: 0.580000
Bias: 0.600000
 
IV calculation completed.
No memory report in this platform.
Elapsed process time = 1.97 sec.


Could not find the Attribute
"AlGaAs.BandModel.Cgam.unstrnd_cubic_cell_length"
in the database. 
Needed for NEMO 3-D but not for NEMO 1-D
Return a zero value.
Could not find the Attribute
"AlGaAs.PhysChem.strain_alpha"
in the database. 
Needed for NEMO 3-D but not for NEMO 1-D
Return a zero value.
Could not find the Attribute
"AlGaAs.PhysChem.strain_beta"
in the database. 
Needed for NEMO 3-D but not for NEMO 1-D
Return a zero value.


(0,
 b'Could not find the Attribute\n"AlGaAs.BandModel.Cgam.unstrnd_cubic_cell_length"\nin the database. \nNeeded for NEMO 3-D but not for NEMO 1-D\nReturn a zero value.\nCould not find the Attribute\n"AlGaAs.PhysChem.strain_alpha"\nin the database. \nNeeded for NEMO 3-D but not for NEMO 1-D\nReturn a zero value.\nCould not find the Attribute\n"AlGaAs.PhysChem.strain_beta"\nin the database. \nNeeded for NEMO 3-D but not for NEMO 1-D\nReturn a zero value.\n',
 b'system_start WARNING: The NEMOHOME environmental variable is not set\nNEMO will assume "/usr/local/nemo" is the correct home directory.\nNo memory report in this platform.\nOpen File: new_input_deck.nem, \nInitialize the basic device structure arrays.\nUpdate Structural Device Parameters and Initialize Arrays\nCopy the material information to the individual monolayers.\nCopy Material Parameters into Device Arrays\nFinished Device Structure Update\nProcess Simulation Parameter Window: Calculation Mode\nRead and Initialize Simulat

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [51]:
# This generates all the outputs and stores them to the data base
# It uses the file names that will be generated by NEMO1D by default 

IVdata = read_data_from_json_file(IV_file)
IVdata=dict(IVdata)
banddata = read_data_from_json_file("/home/toro0/NEMO1D-RTD/simtool/test_AlGaAs_TFQ_Cgam.nd_bnd")

db.save('Band', banddata)


#rIVdata_dict = {'x': IVdata[0], 'y': IVdata[1]} # Wont be used in final deployment 
db.save('IV', IVdata)

Error parsing JSON file: Extra data: line 28 column 6 (char 434)


TypeError: 'NoneType' object is not iterable

: 

In [ ]:
# This cell is being used to test nothing will be transfered to final version 

import numpy as np
import plotly.graph_objects as go
from ipywidgets import widgets, VBox, HBox, Tab, Layout, Button, ButtonStyle
from IPython.display import display, clear_output
import copy

# Template for the plots
layout_template = {
    'title': {
        'font': {
            'family': 'Courier New, monospace',
            'size': 12,
            'color': 'Black'
        }      
    },
    'margin': {
        't': 80,
        'b': 80,
        'r': 100,
        'l': 100
    },
    'showlegend': True,
    'font': {
        'family': 'Courier New, monospace',
        'size': 24,
        'color': 'Black'
    },
    'yaxis': {   
         'title': ""
    },
    'xaxis': {
        'title': ""
    },
    'legend': {
        'orientation': 'h',
        'yanchor': 'bottom',
        'y': 1.0,
        'xanchor': 'right',
        'x': 1
    },
    'template': 'presentation'
}

def update_plot(text_values):
    """Function to update the plot dynamically based on text box inputs."""
    x = [1, 2, 3]  # x-coordinates
    y = [1, 2, 3]  # y-coordinates

    # Clear previous output
    clear_output(wait=True)

    # Create the figure
    fig = go.Figure()

    # Add horizontal lines and text annotations based on text box values
    for xi, yi, text in zip(x, y, text_values):
        fig.add_trace(go.Scatter(
            x=[xi - 0.5, xi + 0.5],
            y=[yi, yi],
            mode='lines',
            line=dict(color='blue'),
            name=f'Line at y={yi}'
        ))
        fig.add_trace(go.Scatter(
            x=[xi],
            y=[yi + 0.2],
            mode='text',
            text=[text],
            textposition='top center',
            showlegend=False
        ))

    # Update layout using the template
    layout = copy.deepcopy(layout_template)
    layout['title']['text'] = "Discrete Horizontal Lines Plot"
    layout['xaxis']['title'] = "X-axis"
    layout['yaxis']['title'] = "Y-axis"
    layout['xaxis'].update(tickmode='array', tickvals=x, range=[0, 4])
    layout['yaxis'].update(tickmode='array', tickvals=y, range=[0, 4])
    
    fig.update_layout(layout)
    fig.show()

    # Re-display widgets
    create_widget()

def create_widget():
    """Function to create widgets for the plot."""
    # Initial text values for annotations
    initial_texts = [f'y={i}' for i in [1, 2, 3]]

    # Create text boxes for each line
    text_boxes = [widgets.Text(value=text, description=f'Line {i+1}:') for i, text in enumerate(initial_texts)]

    # Button to update the plot
    update_button = widgets.Button(description="Update Plot")

    # Event handler for the button
    def on_update_button_click(b):
        text_values = [tb.value for tb in text_boxes]
        update_plot(text_values)

    update_button.on_click(on_update_button_click)

    # Arrange widgets in a vertical layout
    widget_box = VBox(text_boxes + [update_button])
    display(widget_box)

# Simulate Button
simulate_button = Button(
    description="Simulate",
    tooltip='Run simulation',
    icon='check',
    style=ButtonStyle(button_color='lightblue'),
    layout=Layout(width='150px')
)

# Top Tabs
top_tabs = Tab(layout=Layout(width='100%', height='300px'))

# Define contents of the top tabs
basic_tab = VBox([widgets.Label("Basic Tab Content")])
multiscale_domains_tab = VBox([widgets.Label("Multiscale Domains Content")])
advanced_tab = VBox([widgets.Label("Advanced Content")])
resonance_finder_tab = VBox([widgets.Label("Resonance Finder Content")])
device_tab = VBox([widgets.Label("Device Content")])

top_tabs.children = [basic_tab, multiscale_domains_tab, advanced_tab, resonance_finder_tab, device_tab]
top_tabs.set_title(0, "Basic")
top_tabs.set_title(1, "Multiscale Domains")
top_tabs.set_title(2, "Advanced")
top_tabs.set_title(3, "Resonance Finder")
top_tabs.set_title(4, "Device")

# Bottom Tabs
bottom_tabs = Tab(layout=Layout(width='100%', height='200px'))
mole_tab = VBox([widgets.Label("Mole Tab Content")])
doping_tab = VBox([widgets.Label("Doping Tab Content")])

bottom_tabs.children = [mole_tab, doping_tab]
bottom_tabs.set_title(0, "Mole")
bottom_tabs.set_title(1, "Doping")

# Main Layout
layout = VBox([
    simulate_button,
    top_tabs,
    bottom_tabs
], layout=Layout(width='100%', height='700px'))

# Display the final layout
display(layout)

# Initialize plot
create_widget()
update_plot(["y=1", "y=2", "y=3"])

